In [32]:
from sqlalchemy import create_engine, inspect
engine = create_engine('sqlite+pysqlite://///workspaces/D2I-Jupyter-Notebook-Tools/SQL_training/chinook.db', echo=True)
# comment
connection = engine.connect()
inspection = inspect(engine)
inspection.get_table_names()

2023-09-21 13:35:22,801 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-21 13:35:22,802 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-09-21 13:35:22,803 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-21 13:35:22,805 INFO sqlalchemy.engine.Engine ROLLBACK


['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoice_items',
 'invoices',
 'media_types',
 'playlist_track',
 'playlists',
 'tracks']

from sqlalchemy import MetaData, Table 


metadata = MetaData()

tracks = Table('tracks', metadata, autoload_with=engine)

#print(tracks.name)

print(tracks.c.keys())

#print(repr(tracks))

In [33]:
from sqlalchemy import text 

with connection as con:
    stmt = 'SELECT * FROM tracks'
    results_proxy = con.execute(text(stmt))
    results = results_proxy.fetchall()

first_row = results[:10]
for row in first_row:
    print(row.Name)


2023-09-21 13:35:27,464 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-21 13:35:27,466 INFO sqlalchemy.engine.Engine SELECT * FROM tracks
2023-09-21 13:35:27,468 INFO sqlalchemy.engine.Engine [generated in 0.00476s] ()
2023-09-21 13:35:27,482 INFO sqlalchemy.engine.Engine ROLLBACK
For Those About To Rock (We Salute You)
Balls to the Wall
Fast As a Shark
Restless and Wild
Princess of the Dawn
Put The Finger On You
Let's Get It Up
Inject The Venom
Snowballed
Evil Walks


In [38]:
from sqlalchemy import select

with engine.connect() as con:
    stmt = select(tracks)
    results_proxy = con.execute(stmt)
    results = results_proxy.fetchmany(size=10)
print(results)

print(stmt)

2023-09-21 13:37:27,936 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-21 13:37:27,938 INFO sqlalchemy.engine.Engine SELECT tracks."TrackId", tracks."Name", tracks."AlbumId", tracks."MediaTypeId", tracks."GenreId", tracks."Composer", tracks."Milliseconds", tracks."Bytes", tracks."UnitPrice" 
FROM tracks
2023-09-21 13:37:27,939 INFO sqlalchemy.engine.Engine [cached since 96.86s ago] ()
2023-09-21 13:37:27,940 INFO sqlalchemy.engine.Engine ROLLBACK
[(1, 'For Those About To Rock (We Salute You)', 1, 1, 1, 'Angus Young, Malcolm Young, Brian Johnson', 343719, 11170334, Decimal('0.99')), (2, 'Balls to the Wall', 2, 2, 1, None, 342562, 5510424, Decimal('0.99')), (3, 'Fast As a Shark', 3, 2, 1, 'F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman', 230619, 3990994, Decimal('0.99')), (4, 'Restless and Wild', 3, 2, 1, 'F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman', 252051, 4331779, Decimal('0.99')), (5, 'Princess of the Dawn', 3, 2, 1, 'Deaffy & R.A. Smith-Diese

In [40]:
from sqlalchemy import or_

with engine.connect() as con:
    stmt = select(tracks)
    stmt = stmt.where(or_(tracks.columns.AlbumId == 1,
                          tracks.columns.AlbumId == 2))
    results = con.execute(stmt).fetchall()
    
print(results)


2023-09-21 13:47:35,970 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-21 13:47:35,972 INFO sqlalchemy.engine.Engine SELECT tracks."TrackId", tracks."Name", tracks."AlbumId", tracks."MediaTypeId", tracks."GenreId", tracks."Composer", tracks."Milliseconds", tracks."Bytes", tracks."UnitPrice" 
FROM tracks 
WHERE tracks."AlbumId" = ? OR tracks."AlbumId" = ?
2023-09-21 13:47:35,974 INFO sqlalchemy.engine.Engine [generated in 0.00326s] (1, 2)
2023-09-21 13:47:35,975 INFO sqlalchemy.engine.Engine ROLLBACK
[(1, 'For Those About To Rock (We Salute You)', 1, 1, 1, 'Angus Young, Malcolm Young, Brian Johnson', 343719, 11170334, Decimal('0.99')), (6, 'Put The Finger On You', 1, 1, 1, 'Angus Young, Malcolm Young, Brian Johnson', 205662, 6713451, Decimal('0.99')), (7, "Let's Get It Up", 1, 1, 1, 'Angus Young, Malcolm Young, Brian Johnson', 233926, 7636561, Decimal('0.99')), (8, 'Inject The Venom', 1, 1, 1, 'Angus Young, Malcolm Young, Brian Johnson', 210834, 6852860, Decimal('0.99')), (9, 'S

In [44]:
with engine.connect() as con:
    stmt = select(tracks).order_by(tracks.columns.Composer.desc())
    results = con.execute(stmt).fetchmany(size=10)
print(results)

2023-09-21 13:52:49,657 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-21 13:52:49,658 INFO sqlalchemy.engine.Engine SELECT tracks."TrackId", tracks."Name", tracks."AlbumId", tracks."MediaTypeId", tracks."GenreId", tracks."Composer", tracks."Milliseconds", tracks."Bytes", tracks."UnitPrice" 
FROM tracks ORDER BY tracks."Composer" DESC
2023-09-21 13:52:49,659 INFO sqlalchemy.engine.Engine [generated in 0.00217s] ()
2023-09-21 13:52:49,662 INFO sqlalchemy.engine.Engine ROLLBACK
[(817, 'Lick It Up', 66, 1, 1, 'roger glover', 240274, 7792604, Decimal('0.99')), (819, 'Talk About Love', 66, 1, 1, 'roger glover', 247823, 8072171, Decimal('0.99')), (820, 'Time To Kill', 66, 1, 1, 'roger glover', 351033, 11354742, Decimal('0.99')), (821, 'Ramshackle Man', 66, 1, 1, 'roger glover', 334445, 10874679, Decimal('0.99')), (822, 'A Twist In The Tail', 66, 1, 1, 'roger glover', 257462, 8413103, Decimal('0.99')), (824, 'Solitaire', 66, 1, 1, 'roger glover', 282226, 9157021, Decimal('0.99')), (82

In [46]:
from sqlalchemy import func

with engine.connect() as con:
    stmt = select(func.sum(tracks.columns.Milliseconds))
    results = con.execute(stmt).scalar()
print(results)


2023-09-21 13:58:54,801 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-21 13:58:54,802 INFO sqlalchemy.engine.Engine SELECT sum(tracks."Milliseconds") AS sum_1 
FROM tracks
2023-09-21 13:58:54,803 INFO sqlalchemy.engine.Engine [cached since 9.394s ago] ()
2023-09-21 13:58:54,804 INFO sqlalchemy.engine.Engine ROLLBACK
1378778040
